In [16]:
# load modules in other files
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# autoreload self-defined modules
%load_ext autoreload
%autoreload 2

from __future__ import unicode_literals, print_function, division

import os
import time
import argparse

import tensorflow as tf
import torch
from model import Model
from torch.nn.utils import clip_grad_norm_

from torch.optim import Adagrad

from data_util import utils
from data_util import config
from data_util.batcher import Batcher
from data_util.data import Vocab
from data_util.utils import calc_running_avg_loss
from train_util import get_input_from_batch, get_output_from_batch

from data_util import data

from torch.autograd import Variable

use_cuda = config.use_gpu and torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from train import Train

In [4]:
train_processor = Train()

max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: 16-years-old

INFO:tensorflow:Bucket queue size: 0, Input queue size: 0



## Get the batch

In [5]:
v = train_processor.vocab

In [6]:
iter, running_avg_loss = train_processor.setup_train(None)

In [7]:
batch = train_processor.batcher.next_batch()

In [8]:
batch

In [9]:
# device = utils.get_devices(config.gpu_ids)
device = torch.device("cuda:6,7")

In [10]:
enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_1, coverage = get_input_from_batch(batch, use_cuda, device)

dec_batch, dec_padding_mask, max_dec_len, dec_lens_var, target_batch = get_output_from_batch(batch, use_cuda, device)

In [11]:
print(enc_batch.shape)
print(enc_batch)
print(enc_lens.shape)
print(enc_lens)

torch.Size([32, 400])
tensor([[  55,  127,   53,  ...,   90,  409,   35],
        [5620,   55,  127,  ...,    6,  695,  760],
        [  55,  127,   53,  ...,  384,    4,    5],
        ...,
        [  55,  127,   53,  ...,    1,    1,    1],
        [6140,    6, 1045,  ...,    1,    1,    1],
        [7692,    6, 3006,  ...,    1,    1,    1]], device='cuda:6')
torch.Size([32])
tensor([400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 343, 313, 299, 191,
        189, 184, 184, 176], device='cuda:6', dtype=torch.int32)


In [21]:
enc_lens = Variable(torch.from_numpy(enc_lens.cpu().numpy())).int().to(device)
enc_lens

tensor([400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 343, 313, 299, 191,
        189, 184, 184, 176], device='cuda:6', dtype=torch.int32)

In [ ]:
encoder_outputs, encoder_feature, encoder_hidden = train_processor.model.encoder(enc_batch, enc_lens)
# s_t_1 = train_processor.model.reduce_state(encoder_hidden)

In [ ]:
enc_batch

In [55]:
encoder = train_processor.model.encoder.cpu()

In [44]:
enc_lens.tolist()

[400, 400, 400, 400, 400, 343, 184, 176]

In [28]:
from train_util import get_rbm_input_from_batch
docs_word_count, docs_lens = get_rbm_input_from_batch(batch, v, True, device)
docs_word_count

tensor([[7, 0, 2,  ..., 0, 0, 0],
        [2, 3, 1,  ..., 0, 0, 0],
        [3, 0, 0,  ..., 0, 0, 0],
        ...,
        [5, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]], device='cuda:6', dtype=torch.int32)

In [29]:
docs_lens

tensor([180, 177, 184, 199, 181, 193, 170, 178, 178, 166, 193, 161, 151, 180,
        191, 192, 187, 190, 192, 180, 158, 195, 168, 209, 177, 145, 152,  88,
         86,  79, 100,  90], device='cuda:6')

In [25]:
rbm_input.sum(1)

tensor([180, 177, 184, 199, 181, 193, 170, 178, 178, 166, 193, 161, 151, 180,
        191, 192, 187, 190, 192, 180, 158, 195, 168, 209, 177, 145, 152,  88,
         86,  79, 100,  90], device='cuda:6')

In [151]:
print([v.id2word(i) for i in enc_batch[0, :].cpu().numpy()])
print(' '.join([v.id2word(i) for i in enc_batch[0, :].cpu().numpy()]))

['-lrb-', 'cnn', '-rrb-', '--', 'a', 'continental', 'airlines', 'jetliner', 'flew', 'with', 'one', 'engine', 'powered', 'by', 'a', 'mix', 'of', 'oils', 'from', 'algae', 'and', '[UNK]', 'plants', 'wednesday', 'in', 'what', 'the', 'plane', "'s", 'manufacturer', 'called', 'an', '``', 'outstanding', "''", 'test', 'flight', '.', 'continental', 'airlines', 'is', 'the', 'third', 'commercial', 'carrier', 'to', 'test', 'biofuels', 'in', 'their', 'planes', '.', 'the', 'boeing', '737', 'burned', 'less', 'fuel', 'in', 'the', '[UNK]', 'engine', 'than', 'a', 'conventionally', 'powered', 'engine', ',', 'and', 'the', 'two', 'performed', '[UNK]', 'during', 'the', 'one-hour', ',', '40-minute', 'flight', ',', 'boeing', 'spokesman', 'terrance', 'scott', 'said', '.', '``', 'there', 'was', 'no', 'hesitation', 'at', 'all', ',', "''", 'scott', 'said', '.', '``', 'if', 'you', 'had', 'to', 'grade', 'it', 'on', 'a', 'scale', 'of', 'one', 'to', '10', ',', 'it', 'would', 'be', 'an', '11', '.', "''", 'wednesday', "

In [155]:
print([v.id2word(i) for i in enc_batch[7, :].cpu().numpy()])

['athens', ',', 'greece', '-lrb-', 'cnn', '-rrb-', '--', 'u.s.', 'secretary', 'of', 'state', 'hillary', 'clinton', 'has', 'some', 'apple', 'picking', 'to', 'do', '.', 'clinton', 'placed', 'a', 'bet', 'on', 'the', 'women', "'s", 'world', 'cup', 'finals', 'with', 'her', 'japanese', 'counterpart', '--', 'new', 'york', 'apples', 'for', 'japanese', 'pears', '.', 'the', 'japanese', 'women', "'s", 'soccer', 'team', 'won', 'the', 'world', 'cup', 'sunday', 'after', 'defeating', 'the', 'united', 'states', 'on', 'penalty', 'kicks', '.', 'clinton', 'alluded', 'to', 'the', 'bet', 'in', 'a', 'phone', 'call', 'she', 'placed', 'to', 'the', 'u.s.', 'team', 'to', 'wish', 'them', 'luck', 'ahead', 'of', 'the', 'game', '.', 'at', 'the', 'time', ',', 'she', 'did', "n't", 'say', 'what', 'the', 'bet', 'with', 'japan', "'s", 'foreign', 'minister', '[UNK]', '[UNK]', 'entailed', '.', '``', 'i', 'am', 'sorry', 'i', 'ca', "n't", 'be', 'there', 'with', 'you', 'tonight', ',', 'but', 'i', 'sent', 'my', 'very', 'best'

In [ ]:
docs = []
for i in range(enc_batch.shape[0]):
    doc = [v.id2word(i) for i in enc_batch[7, :].cpu().numpy()]
    

In [162]:
# read the top 15k words into a list
with open("/home/yang6367/gitrepos/cnn-dailymail/top15k_words.txt", "r") as f:
    top15k_words = [word.rstrip() for word in f.readlines()]
top15k_words

INFO:tensorflow:Bucket queue size: 100, Input queue size: 800


['say',
 'will',
 'year',
 'one',
 "n't",
 'take',
 'make',
 'go',
 'time',
 'people',
 'new',
 'also',
 'tell',
 'get',
 'two',
 'last',
 'mr',
 'just',
 'police',
 'find',
 'day',
 'first',
 'leave',
 'come',
 'use',
 'work',
 'now',
 'see',
 'home',
 'show',
 'know',
 'like',
 'family',
 'child',
 'think',
 'back',
 'world',
 'report',
 'give',
 'state',
 'call',
 'right',
 'woman',
 'want',
 'week',
 'look',
 'include',
 'life',
 'three',
 'late',
 'month',
 'cnn',
 'even',
 'help',
 'good',
 'picture',
 'court',
 'may',
 'claim',
 'many',
 'country',
 'city',
 'around',
 'way',
 'house',
 'man',
 'play',
 'since',
 'try',
 'kill',
 'need',
 'win',
 'accord',
 'video',
 'school',
 'add',
 'start',
 'u.s.',
 'government',
 'still',
 'group',
 'president',
 'part',
 'become',
 'team',
 'former',
 'death',
 'attack',
 'place',
 'game',
 'case',
 'early',
 'million',
 'face',
 'official',
 'u',
 'follow',
 'another',
 'charge',
 'high',
 'well',
 'unite',
 'force',
 'live',
 'shoot',
 

In [165]:
from collections import Counter, OrderedDict

In [195]:
enc_batch.cpu().tolist()

[[55,
  127,
  53,
  59,
  8,
  9109,
  1910,
  22628,
  2777,
  22,
  56,
  2749,
  6880,
  29,
  8,
  3892,
  10,
  16393,
  31,
  15476,
  9,
  0,
  3345,
  347,
  11,
  87,
  5,
  783,
  12,
  6282,
  195,
  41,
  36,
  4260,
  35,
  830,
  657,
  4,
  9109,
  1910,
  19,
  5,
  519,
  2192,
  4953,
  7,
  830,
  31143,
  11,
  49,
  3084,
  4,
  5,
  4356,
  19474,
  3451,
  462,
  2069,
  11,
  5,
  0,
  2749,
  79,
  8,
  43122,
  6880,
  2749,
  6,
  9,
  5,
  72,
  2446,
  0,
  115,
  5,
  22646,
  6,
  32303,
  657,
  6,
  4356,
  397,
  39601,
  1641,
  23,
  4,
  36,
  67,
  13,
  90,
  17945,
  26,
  71,
  6,
  35,
  1641,
  23,
  4,
  36,
  86,
  58,
  43,
  7,
  4176,
  21,
  17,
  8,
  2690,
  10,
  56,
  7,
  280,
  6,
  21,
  64,
  34,
  41,
  624,
  4,
  35,
  347,
  12,
  830,
  6,
  2468,
  31,
  9109,
  12,
  7114,
  11,
  2853,
  6,
  1017,
  6,
  13,
  5,
  89,
  29,
  8,
  150,
  4953,
  9,
  5,
  89,
  7,
  255,
  8,
  9461,
  10,
  16393,
  31,
  15476,
  9,


In [199]:
docs = [[v.id2word(i) for i in doc_ids ]for doc_ids in enc_batch.cpu().tolist()]
# remove stopwords, meaningless words, and do lemmatization
docs = [utils.lemmatize(utils.clean(doc)) for doc in docs]
print(docs)

[['continental', u'airline', 'jetliner', u'fly', 'one', 'engine', u'power', 'mix', u'oil', 'algae', u'plant', 'wednesday', 'plane', 'manufacturer', u'call', 'outstanding', 'test', 'flight', 'continental', u'airline', 'third', 'commercial', 'carrier', 'test', 'biofuels', u'plane', 'boeing', '737', u'burn', u'le', 'fuel', 'engine', 'conventionally', u'power', 'engine', 'two', u'perform', 'one-hour', '40-minute', 'flight', 'boeing', 'spokesman', 'terrance', 'scott', u'say', 'hesitation', 'scott', u'say', 'grade', 'scale', 'one', '10', '11', 'wednesday', 'test', u'conduct', 'continental', 'hub', 'houston', 'texas', 'first', 'u.s.', 'carrier', 'first', 'use', 'blend', u'oil', 'algae', 'scott', u'say', 'one', 'engine', u'run', '50-50', 'mix', 'combination', 'conventional', 'jet', 'fuel', u'run', 'entirely', 'conventional', 'fuel', '737', 'one', 'common', 'aircraft', 'service', 'u.s.', u'market', 'crew', 'two', u'pilot', 'engineer', 'put', 'jet', 'series', u'exercise', u'include', u'measure',

In [210]:
## step 1: get the cleaned word list for each document
# convert the indices to words
docs = [[v.id2word(i) for i in doc_ids ]for doc_ids in enc_batch.cpu().tolist()]
# remove stopwords, meaningless words, and do lemmatization
docs = [utils.lemmatize(utils.clean(doc)) for doc in docs]
# filter out the words that are not in the top15k
docs = [utils.keep_top15k(doc) for doc in docs]

## step 2: transform the word list into a word-count vector
docs_word_count = [utils.get_word_count(doc) for doc in docs]
# transform the list to a tensor
out = Variable(torch.Tensor(docs_word_count).int())
print(out.shape)
out

# # initialize the orderedDict
# ordered_dict = OrderedDict((w, 0) for w in top15k_words)
# # update the orderedDict
# for word, count in dict(Counter(doc)).items():
#     ordered_dict[word] = count
# # extract the word count as a tensor
# Variable(torch.Tensor(ordered_dict.values()).int())

torch.Size([8, 15000])


tensor([[7, 0, 2,  ..., 0, 0, 0],
        [2, 3, 1,  ..., 0, 0, 0],
        [3, 0, 0,  ..., 0, 0, 0],
        ...,
        [7, 1, 1,  ..., 0, 0, 0],
        [5, 0, 1,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [205]:
docs

[['continental',
  u'airline',
  'jetliner',
  u'fly',
  'one',
  'engine',
  u'power',
  'mix',
  u'oil',
  'algae',
  u'plant',
  'wednesday',
  'plane',
  'manufacturer',
  u'call',
  'outstanding',
  'test',
  'flight',
  'continental',
  u'airline',
  'third',
  'commercial',
  'carrier',
  'test',
  'biofuels',
  u'plane',
  'boeing',
  '737',
  u'burn',
  u'le',
  'fuel',
  'engine',
  'conventionally',
  u'power',
  'engine',
  'two',
  u'perform',
  'one-hour',
  '40-minute',
  'flight',
  'boeing',
  'spokesman',
  'terrance',
  'scott',
  u'say',
  'hesitation',
  'scott',
  u'say',
  'grade',
  'scale',
  'one',
  '10',
  '11',
  'wednesday',
  'test',
  u'conduct',
  'continental',
  'hub',
  'houston',
  'texas',
  'first',
  'u.s.',
  'carrier',
  'first',
  'use',
  'blend',
  u'oil',
  'algae',
  'scott',
  u'say',
  'one',
  'engine',
  u'run',
  '50-50',
  'mix',
  'combination',
  'conventional',
  'jet',
  'fuel',
  u'run',
  'entirely',
  'conventional',
  'fuel',

In [218]:
## step 1: get the cleaned word list for each document
# convert the indices to words
doc = [v.id2word(i) for i in enc_batch[1, :].cpu().numpy()]
# remove stopwords, meaningless words, and do lemmatization
doc = utils.clean(doc)
doc = utils.lemmatize(doc)
# filter out the words that are not in the top15k
doc = [w for w in doc if w in top15k_words]

## step 2: transform the word list into a word-count vector
# initialize the orderedDict
ordered_dict = OrderedDict((w, 0) for w in top15k_words)
# update the orderedDict
for word, count in dict(Counter(doc)).items():
    ordered_dict[word] = count
# extract the word count as a tensor
Variable(torch.Tensor(ordered_dict.values()).int())


tensor([2, 3, 1,  ..., 0, 0, 0], dtype=torch.int32)

In [170]:
from collections import OrderedDict


OrderedDict([('say', 0),
             ('will', 0),
             ('year', 0),
             ('one', 0),
             ("n't", 0),
             ('take', 0),
             ('make', 0),
             ('go', 0),
             ('time', 0),
             ('people', 0),
             ('new', 0),
             ('also', 0),
             ('tell', 0),
             ('get', 0),
             ('two', 0),
             ('last', 0),
             ('mr', 0),
             ('just', 0),
             ('police', 0),
             ('find', 0),
             ('day', 0),
             ('first', 0),
             ('leave', 0),
             ('come', 0),
             ('use', 0),
             ('work', 0),
             ('now', 0),
             ('see', 0),
             ('home', 0),
             ('show', 0),
             ('know', 0),
             ('like', 0),
             ('family', 0),
             ('child', 0),
             ('think', 0),
             ('back', 0),
             ('world', 0),
             ('report', 0),
             

In [178]:
for word, count in dict(Counter(doc)).items():
    ordered_dict[word] = count
ordered_dict

OrderedDict([('say', 1),
             ('will', 0),
             ('year', 0),
             ('one', 1),
             ("n't", 2),
             ('take', 0),
             ('make', 0),
             ('go', 0),
             ('time', 1),
             ('people', 0),
             ('new', 1),
             ('also', 0),
             ('tell', 1),
             ('get', 0),
             ('two', 0),
             ('last', 0),
             ('mr', 0),
             ('just', 0),
             ('police', 0),
             ('find', 0),
             ('day', 0),
             ('first', 0),
             ('leave', 0),
             ('come', 0),
             ('use', 0),
             ('work', 0),
             ('now', 0),
             ('see', 0),
             ('home', 0),
             ('show', 0),
             ('know', 0),
             ('like', 0),
             ('family', 0),
             ('child', 0),
             ('think', 0),
             ('back', 0),
             ('world', 2),
             ('report', 0),
             

In [188]:
Variable(torch.Tensor(ordered_dict.values()).int())

tensor([1, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

## Remove unrelevant words

In [17]:
input_gen = train_processor.batcher.text_generator(data.example_generator(config.train_data_path, False))

In [18]:
(article, abstract) = input_gen.next()

In [19]:
article

'lagos , nigeria -lrb- cnn -rrb- -- investigators in nigeria say they have taken 13 people into custody after a mob attacked four students and burned them alive -- killings that were videotaped and posted online . a gruesome video on youtube that showed the killings led to several arrests , police say . the video shows several men on the dirt , writhing in pain . some are bloodied , and they have tires shoved over their heads . read more : why are nigerians numb to slaughter ? investigators have not yet determined why the mob killed the men , said ben ugwuegbulam , a police spokesman in rivers state , in southern nigeria . the attack happened friday . mob justice is common in nigeria , with some people choosing to take the law into their own hands because police are either unresponsive or unwilling to patrol high-risk areas after dark . the posting has drawn comments from some who say that police should have done more to investigate the attack before the video was posted . read more : 

In [34]:
a = [v.id2word(i) for i in enc_batch[7, :].cpu().numpy()]
print(a)


['athens', ',', 'greece', '-lrb-', 'cnn', '-rrb-', '--', 'u.s.', 'secretary', 'of', 'state', 'hillary', 'clinton', 'has', 'some', 'apple', 'picking', 'to', 'do', '.', 'clinton', 'placed', 'a', 'bet', 'on', 'the', 'women', "'s", 'world', 'cup', 'finals', 'with', 'her', 'japanese', 'counterpart', '--', 'new', 'york', 'apples', 'for', 'japanese', 'pears', '.', 'the', 'japanese', 'women', "'s", 'soccer', 'team', 'won', 'the', 'world', 'cup', 'sunday', 'after', 'defeating', 'the', 'united', 'states', 'on', 'penalty', 'kicks', '.', 'clinton', 'alluded', 'to', 'the', 'bet', 'in', 'a', 'phone', 'call', 'she', 'placed', 'to', 'the', 'u.s.', 'team', 'to', 'wish', 'them', 'luck', 'ahead', 'of', 'the', 'game', '.', 'at', 'the', 'time', ',', 'she', 'did', "n't", 'say', 'what', 'the', 'bet', 'with', 'japan', "'s", 'foreign', 'minister', '[UNK]', '[UNK]', 'entailed', '.', '``', 'i', 'am', 'sorry', 'i', 'ca', "n't", 'be', 'there', 'with', 'you', 'tonight', ',', 'but', 'i', 'sent', 'my', 'very', 'best'

In [38]:
lemmatize(clean(remove_stopwords(a)))

['athens',
 'greece',
 'u.s.',
 'secretary',
 'state',
 'hillary',
 'clinton',
 'apple',
 u'pick',
 'clinton',
 u'place',
 'bet',
 u'woman',
 'world',
 'cup',
 u'final',
 'japanese',
 'counterpart',
 'new',
 'york',
 u'apple',
 'japanese',
 u'pear',
 'japanese',
 u'woman',
 'soccer',
 'team',
 u'win',
 'world',
 'cup',
 'sunday',
 u'defeat',
 u'unite',
 u'state',
 'penalty',
 u'kick',
 'clinton',
 u'allude',
 'bet',
 'phone',
 'call',
 u'place',
 'u.s.',
 'team',
 'wish',
 'luck',
 'ahead',
 'game',
 'time',
 "n't",
 'say',
 'bet',
 'japan',
 'foreign',
 'minister',
 u'entail',
 'sorry',
 'ca',
 "n't",
 'tonight',
 u'send',
 'best',
 'good',
 'luck',
 'charm',
 'one',
 'extra',
 u'bite',
 'luck',
 'daughter',
 'chelsea',
 'clinton',
 u'tell',
 'daughter',
 'chelsea',
 'clinton',
 'among',
 u'attend',
 'sunday',
 'game',
 'frankfurt',
 'part',
 'delegation',
 u'include',
 'jill',
 'biden',
 'wife',
 'u.s.',
 'vice',
 'president']

## Process the vocab

In [65]:
with open(config.vocab_path, 'r') as vocab_f:
    i = 0
    for line in vocab_f:
        pieces = line.split()
        w = pieces[0]
        print(pieces)
        i += 1
        if i > 5:
            break


['.', '12172181']
['the', '11896296']
[',', '9609022']
['to', '5751102']
['a', '5100569']
['and', '4892246']
INFO:tensorflow:Bucket queue size: 100, Input queue size: 800
INFO:tensorflow:Bucket queue size: 100, Input queue size: 800


In [131]:
## step 1: load the stopwords provided by Stanford CoreNLP
f = open("/home/yang6367/gitrepos/cnn-dailymail/corenlp-stopwords.txt")
stopwords = f.read().split('\n')
new_stopwords = ['!!!', '!!!!', "'''", "**", "***", "****", "*****", "******", "*******", "--", "-rcb-", "/", ]
stopwords = stopwords + new_stopwords

## step 2: create a dictionary containing the lemmatized word count
word_to_count = {}
num_words = 0
max_size = 50000
with open(config.vocab_path, 'r') as vocab_f:
    for line in vocab_f:
        pieces = line.split()
        try:
            # if the word is not a stopword, then add the count to the dictionary
            if w not in stopwords:
                w = utils.lem(pieces[0]) # lemmatize the word
                if w in word_to_count:  # if already in, then add up the count
                    word_to_count[w] += int(pieces[1])
                else:
                    word_to_count[w] = int(pieces[1])
                    num_words += 1
        except:
            print(pieces)
        if num_words >= max_size:
            break

## step 3: get top15k words
# sort the dictionary by the count
word_to_count_sorted = sorted(word_to_count.items(), key=lambda x: x[1], reverse=True)
# choose the top 15k words
word_to_count_top15k = word_to_count_sorted[:15000]
words_top15k = [key for key, value in word_to_count_top15k]
# write the list of words to a txt file
with open("/home/yang6367/gitrepos/cnn-dailymail/top15k_words.txt", "w") as f:
    f.writelines("%s\n" % word for word in words_top15k)

In [146]:
# read the top 15k words into a list
with open("/home/yang6367/gitrepos/cnn-dailymail/top15k_words.txt", "r") as f:
    words = [word.rstrip() for word in f.readlines()]
words

['wa',
 'say',
 'ha',
 'will',
 'year',
 'one',
 "n't",
 'take',
 'make',
 'go',
 'time',
 'people',
 'new',
 'also',
 'tell',
 'get',
 'two',
 'last',
 'mr',
 'just',
 'police',
 'find',
 'day',
 'first',
 'leave',
 'come',
 'use',
 'work',
 'now',
 'see',
 'home',
 'show',
 'know',
 'like',
 'family',
 'child',
 'think',
 'back',
 'world',
 'report',
 'give',
 'state',
 'call',
 'right',
 'woman',
 'want',
 'week',
 'look',
 'include',
 'life',
 'three',
 'late',
 'month',
 'cnn',
 'even',
 'help',
 'good',
 'picture',
 'court',
 'may',
 'claim',
 'many',
 'country',
 'city',
 'around',
 'way',
 'house',
 'man',
 'play',
 'since',
 'try',
 'kill',
 'need',
 'win',
 'accord',
 'video',
 'school',
 'add',
 'start',
 'u.s.',
 'government',
 'still',
 'group',
 'president',
 'part',
 'become',
 'team',
 'former',
 'death',
 'attack',
 'place',
 'game',
 'case',
 'early',
 'million',
 'face',
 'official',
 'u',
 'follow',
 'another',
 'charge',
 'high',
 'well',
 'unite',
 'force',
 'live